# U:FF 2025 Tier Prediction (NN)

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
def split_image_grid(image_path, rows, cols, label):
    img = Image.open(image_path)
    width, height = img.size
    tile_width = width // cols
    tile_height = height // rows

    images = []
    labels = []

    for i in range(rows):
        for j in range(cols):
            box = (j * tile_width, i * tile_height, (j + 1) * tile_width, (i + 1) * tile_height)
            tile = img.crop(box)
            tile = tile.resize((32, 32))  # Resize for uniformity
            images.append(tile)
            labels.append(label)
            
    return images, labels

## Training

In [ ]:
def prepare_data(images):
    X = [np.array(img.convert('L')).flatten() / 255.0 for img in images]  # grayscale + normalization
    return np.array(X)

In [ ]:
sheet_specs = [
    ("tiere/uff-tiere-lieb-1.png", 6, 6, 0),  # label 0 = lieb
    ("tiere/uff tiere-lieb-2.png", 6, 6, 0),
    ("tiere/uff-tiere-böse-1.png", 6, 6, 1),  # label 1 = böse
    ("tiere/uff-tiere-böse-2.png", 6, 6, 1)
]

all_images = []
all_labels = []

for path, rows, cols, label in sheet_specs:
    imgs, labels = split_image_grid(path, rows, cols, label)
    all_images.extend(imgs)
    all_labels.extend(labels)

In [ ]:
all_images[19]

In [ ]:
X = prepare_data(all_images)
y = np.array(all_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()  # for binary classification
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
model = SimpleNN(X_train.shape[1])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    predicted_classes = (predictions > 0.5).float()
    accuracy = (predicted_classes == y_test_tensor).float().mean()
    print(f"Test Accuracy: {accuracy.item():.4f}")

## Prediction

In [ ]:
imgs, labels = split_image_grid("tiere/uff-tiere-gemischt.png", 6, 6, 2)
X_mixed = prepare_data(imgs)

X_mixed_tensor = torch.tensor(X_mixed, dtype=torch.float32)

model.eval()
with torch.no_grad():
    pred_probs = model(X_mixed_tensor)
    pred_labels = (pred_probs > 0.5).float()

predictions = pred_labels.numpy().astype(int).flatten()

In [ ]:
imgs[13]

In [ ]:
predictions[13]

In [ ]:
def draw_predictions_on_tiles(tiles, predictions):
    labeled_tiles = []
    font = ImageFont.load_default()

    for img, pred in zip(tiles, predictions):
        label = "Lieb" if pred == 0 else "Boese"

        img_copy = img.convert("RGBA")
        txt_overlay = Image.new("RGBA", img_copy.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_overlay)

        bbox = draw.textbbox((0, 0), label, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = (img_copy.width - text_width) // 2
        y = (img_copy.height - text_height) // 2

        padding = 4
        background_box = [
            x - padding,
            y - padding,
            x + text_width + padding,
            y + text_height + padding
        ]
        draw.rectangle(background_box, fill=(255, 255, 255, 180))

        draw.text((x, y), label, fill="black", font=font)

        combined = Image.alpha_composite(img_copy, txt_overlay).convert("RGB")
        labeled_tiles.append(combined)

    return labeled_tiles

def save_image_grid(images, rows, cols, output_path):
    w, h = images[0].size
    grid_img = Image.new("RGB", (cols * w, rows * h), "white")

    for i, img in enumerate(images):
        row = i // cols
        col = i % cols
        grid_img.paste(img, (col * w, row * h))

    grid_img.save(output_path)
    print(f"Saved prediction grid to: {output_path}")

labeled = draw_predictions_on_tiles(imgs, predictions)
save_image_grid(labeled, 6, 6, "predicted_grid_nn.png")